In [1]:
import pandas as pd

from src.data import read_data

In [2]:
train = read_data('data/train.jsonl')

In [3]:
train.head()

,id,title,question,context,spoiler
0,0af11f6b-c889-4520-9372-66ba25cb7657,"Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...",[how about that morning we go throw?]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,Hole In Ozone Layer Expected To Make Full Reco...,NASA sets date for full recovery of ozone hole,Hole In Ozone Layer Expected To Make Full Reco...,[2070]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,Intellectual Stimulation Trumps Money For Empl...,This is what makes employees happy -- and it's...,Intellectual Stimulation Trumps Money For Empl...,[intellectual stimulation]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,"‘Follow your passion’ is wrong, here are 7 hab...",Passion is overrated — 7 work habits you need ...,"‘Follow your passion’ is wrong, here are 7 hab...",[Purpose connects us to something bigger and i...
4,31b108a3-c828-421a-a4b9-cf651e9ac859,Revealed: The perfect way to cook rice so that...,The perfect way to cook rice so that it's perf...,Revealed: The perfect way to cook rice so that...,[in a rice cooker]


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3200 non-null   object
 1   title     3200 non-null   object
 2   question  3200 non-null   object
 3   context   3200 non-null   object
 4   spoiler   3200 non-null   object
dtypes: object(5)
memory usage: 125.1+ KB
